In [1]:
import sys
sys.path.append('../../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.ANN import lstmPolicyPredictor, FullyConnected
from Code.envs.GPEnv import PassiveEnv
from wip.Code.train import make_dataset_simple
from Code.SNN import DynNetwork, SequenceWrapper
from Code.ANN import LSTMWrapper, ReLuWrapper
import time
from collections import OrderedDict
from Code import Neurons

In [2]:
#TODO: output reset mechanism

config = {
    'ALPHA': 0.5, #0.7
    'BETA': 0.9, #0.95
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

secondconfig = {
    'ALPHA': 0.7,
    'BETA': 0.5, #0.95
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

heavyside = {
    'ALPHA': 0,
    'BETA': 1, #0.95
    'RESET_ZERO': False,
    'SPIKE_FN': 'ss'
}

outconfig = {
    'ALPHA': 0,
    'BETA': 0,
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

class Selector(nn.Module):
    def __init__(self, params, size):
        super().__init__()
        self.start = params
        self.end = params + size

    def forward(self, x, h):
        return x[:, self.start:self.end], ()

    def get_initial_state(self, batch_size):
        return ()

architecture1 = OrderedDict([
    ('input', [3]),
    ('obs', [2, ['input'], Selector, 0, None]),
    ('probe', [1, ['input'], Selector, 2, None]),
    ('pre_mem', [128, ['obs', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('mem', [64, ['pre_mem'], Neurons.CooldownNeuron, config, nn.Linear]), #CooldownNeuron 32
    ('short_mem', [64, ['pre_mem'], Neurons.CooldownNeuron, secondconfig, nn.Linear]), #32
    ('post_mem', [128, ['probe', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('output', [2, ['post_mem'], Neurons.OutputNeuron, outconfig, nn.Linear]), #OutputNeuron
])

architecture0 = OrderedDict([
    ('input', [3]),
    ('obs', [2, ['input'], Selector, 0, None]),
    ('probe', [1, ['input'], Selector, 2, None]),
    ('pre_mem', [128, ['obs', 'mem'], Neurons.LIFNeuron, outconfig, nn.Linear]),
    ('mem', [128, ['pre_mem'], Neurons.CooldownNeuron, heavyside, nn.Linear]), #CooldownNeuron 32
    ('post_mem', [128, ['probe', 'mem'], Neurons.LIFNeuron, outconfig, nn.Linear]),
    ('output', [2, ['post_mem'], Neurons.OutputNeuron, outconfig, nn.Linear]), #OutputNeuron
])

architecture2 = OrderedDict([
    ('input', [3]),
    ('obs', [2, ['input'], Selector, 0, None]),
    ('probe', [1, ['input'], Selector, 2, None]),
    ('pre_mem', [64, ['obs', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('mem', [128, ['pre_mem'], Neurons.CooldownNeuron, config, nn.Linear]), #CooldownNeuron 32
    ('short_mem', [128, ['pre_mem'], Neurons.CooldownNeuron, secondconfig, nn.Linear]), #32
    ('post_mem', [64, ['probe', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('secondproc', [64, ['post_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('output', [2, ['secondproc'], Neurons.OutputNeuron, outconfig, nn.Linear]), #OutputNeuron
])

architecturelstm0 = OrderedDict([
    ('input', [3]),
    ('obs', [2, ['input'], Selector, 0, None]),
    ('probe', [1, ['input'], Selector, 2, None]),
    ('lstm', [128, ['obs'], LSTMWrapper, None, nn.Linear]),
    ('post_mem', [128, ['probe', 'lstm'], ReLuWrapper, None, nn.Linear]),
    ('output', [2, ['post_mem'], Neurons.OutputNeuron, outconfig, nn.Linear]), #OutputNeuron
])


BATCH_SIZE = 1024
SIM_TIME = 1
MAX_ITER = 50

device = torch.device('cuda')


In [3]:
from Code.Networks import Selector, DynNetwork, SequenceWrapper, OuterWrapper, LSTMWrapper, ReLuWrapper
from Code.NewNeurons import SeqOnlySpike, CooldownNeuron, OutputNeuron

base_config = {
    'ALPHA': 0,
    'BETA': 0,
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

heavyside = {
    'ALPHA': 0,
    'BETA': 1, #0.95
    'RESET_ZERO': False,
    'SPIKE_FN': 'ss'
}

mem_loop = OrderedDict([
    ('input', 2),
    ('pre_mem', [['input', 'output'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['pre_mem'], CooldownNeuron(128, heavyside), nn.Linear]),
])

architecture = OrderedDict([
    ('input', 3),
    ('obs', [['input'], Selector(0, 2), None]),
    ('probe', [['input'], Selector(2, 1), None]),
    ('mem_loop', [['obs'], SequenceWrapper(DynNetwork(mem_loop)), None]),
    ('post_mem', [['probe', 'mem_loop'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['post_mem'], OutputNeuron(2, base_config), nn.Linear]),
])

architecturelstm = OrderedDict([
    ('input', 3),
    ('obs', [['input'], Selector(0, 2), None]),
    ('probe', [['input'], Selector(2, 1), None]),
    ('lstm', [['obs'], LSTMWrapper(2, 128), None]),
    ('post_mem', [['probe', 'lstm'], ReLuWrapper(128), nn.Linear]),
    ('output', [['post_mem'], OutputNeuron(2, base_config), nn.Linear]),
])

#TODO: fix output

model = OuterWrapper(DynNetwork(architecture), BATCH_SIZE, device, False)

In [3]:
device = torch.device('cuda')

env = PassiveEnv(BATCH_SIZE, MAX_ITER, device)
#model = lstmPolicyPredictor(1,32,64)

#model = FullyConnected([1, 128, 128, 1])

model_raw = DynNetwork(architecture, SIM_TIME)
model = SequenceWrapper(model_raw, BATCH_SIZE, device, False)



In [22]:
mse = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=5e-5)#0.000011e-6
#optimizer = optim.Adam(model.model.layers['output_linear'].parameters(), lr=1e-4)#0.000011e-6

In [23]:
start = time.time()

for i in range(10000):
    model.zero_grad()
    inputs, targets = env.getBatch()
    if i%100 == 0:
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
    outputs, _ = model(inputs)
    processed = torch.empty_like(outputs)
    processed[:, :, 1] = outputs[:, :, 1]
    processed[:, :, 0] = torch.sigmoid(outputs[:, :, 0])
    loss = mse(processed, targets)
    #loss = mse(outputs[..., 1], targets[..., 1])
    loss.backward()
    optimizer.step()
    if i%100 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i)

print('Total time: ', time.time()-start)




0.043683312833309174 0.07636509090662003 0
0.042748503386974335 0.07718075066804886 100
0.042002614587545395 0.07323010265827179 200
0.04200999066233635 0.07358777523040771 300
0.041039008647203445 0.07261829078197479 400
0.043303847312927246 0.07408227771520615 500
0.0407387837767601 0.07289744913578033 600
0.04048919677734375 0.072781041264534 700
0.041787758469581604 0.07650204747915268 800
0.03737260401248932 0.06898397952318192 900
0.04042200371623039 0.0725611224770546 1000
0.04019938409328461 0.07072747498750687 1100
0.03956860676407814 0.07087235897779465 1200
0.04184488207101822 0.07328825443983078 1300
0.0388055182993412 0.06901288032531738 1400
0.04098735749721527 0.07384707778692245 1500
0.04019228741526604 0.07399100065231323 1600
0.041445836424827576 0.07105650007724762 1700
0.0401562824845314 0.07046672701835632 1800
0.040979545563459396 0.07318063825368881 1900
0.03995523229241371 0.073030985891819 2000
0.04128078371286392 0.07259466499090195 2100
0.04097726196050644 0.

In [28]:
from matplotlib import pyplot as plt
#model = torch.load('../models/snn_passive3')
%matplotlib


inputs, targets = env.getBatch()
outputs, _ = model(inputs)
plt.scatter(inputs[:, 0, 2].cpu(), targets[:, 0, 1].cpu(), label='Mean_Target')
plt.scatter(inputs[:, 0, 2].cpu(), outputs[:, 0, 1].detach().cpu(), label='Mean')
plt.scatter(inputs[:, 0, 2].cpu(), targets[:, 0, 0].cpu(), label='Var_Target')
plt.scatter(inputs[:, 0, 2].cpu(), torch.sigmoid(outputs[:, 0, 0].cpu()).detach(), label='Var')
plt.legend()


Using matplotlib backend: TkAgg


In [25]:
torch.save(model, '../../models/rsnn_gppred2')


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:354: UserWarning: Couldn't retrieve source code for container of type Selector. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [7]:
obs, targets = env.getBatch()

In [8]:
obs[:, 0]


tensor([[ 0.0725,  1.3078,  0.7469],
        [ 0.7469,  0.1004,  0.1551],
        [ 0.1551,  0.7648,  0.5999],
        [ 0.5999, -0.3369,  0.3660],
        [ 0.3660,  0.6587,  0.2637],
        [ 0.2637,  0.3779,  0.2201],
        [ 0.2201,  0.3814,  0.1194],
        [ 0.1194,  1.0581,  0.8709],
        [ 0.8709,  0.5644,  0.1411],
        [ 0.1411,  0.8821,  0.6016],
        [ 0.6016, -0.3385,  0.0749],
        [ 0.0749,  1.3016,  0.7182],
        [ 0.7182, -0.0277,  0.5491],
        [ 0.5491, -0.1718,  0.2822],
        [ 0.2822,  0.4266,  0.7817],
        [ 0.7817,  0.2554,  0.1489],
        [ 0.1489,  0.8160,  0.9466],
        [ 0.9466,  0.6245,  0.5037],
        [ 0.5037,  0.1100,  0.6078],
        [ 0.6078, -0.3423,  0.3834],
        [ 0.3834,  0.6589,  0.1964],
        [ 0.1964,  0.4742,  0.7783],
        [ 0.7783,  0.2406,  0.3649],
        [ 0.3649,  0.6579,  0.9045],
        [ 0.9045,  0.6172,  0.5368],
        [ 0.5368, -0.1034,  0.7161],
        [ 0.7161, -0.0366,  0.4711],
 

In [9]:
targets[:, 0]


tensor([[ 1.7684e-20,  1.7391e-10],
        [ 5.0578e-01,  9.3009e-01],
        [ 1.1508e-01,  3.4039e-02],
        [ 2.4592e-02, -4.2329e-02],
        [ 6.7668e-01,  4.2292e-01],
        [ 9.8833e-01,  3.8013e-01],
        [ 9.9864e-01,  1.0591e+00],
        [ 2.3511e-01,  1.0906e-01],
        [ 9.9999e-01,  8.8214e-01],
        [ 9.9982e-01, -3.3843e-01],
        [ 9.9999e-01,  1.3017e+00],
        [ 9.8745e-01, -4.8751e-02],
        [ 9.7186e-01, -1.9440e-01],
        [ 9.9988e-01,  4.2651e-01],
        [ 9.9765e-01,  2.5635e-01],
        [ 9.9999e-01,  8.1605e-01],
        [ 8.5181e-01,  5.0005e-01],
        [ 9.9471e-01,  1.1237e-01],
        [ 9.9999e-01, -3.4233e-01],
        [ 9.9987e-01,  6.5898e-01],
        [ 9.9999e-01,  4.7421e-01],
        [ 9.9999e-01,  2.4056e-01],
        [ 9.9999e-01,  6.5790e-01],
        [ 9.9935e-01,  6.1698e-01],
        [ 9.9999e-01, -1.0340e-01],
        [ 9.9999e-01, -3.6590e-02],
        [ 9.9994e-01,  3.2506e-01],
        [ 9.9712e-01,  5.774

In [12]:
#model2 = torch.load('../../models/rsnn_gppred1')


In [15]:
#model2.cpu()

SequenceWrapper(
  (pretrace): DynNetwork(
    (layers): ModuleDict(
      (obs): Selector()
      (probe): Selector()
      (pre_mem_linear): Linear(in_features=130, out_features=128, bias=True)
      (pre_mem): LIFNeuron()
      (mem_linear): Linear(in_features=128, out_features=128, bias=True)
      (mem): CooldownNeuron(
        (elu): ELU(alpha=1.0)
      )
      (post_mem_linear): Linear(in_features=129, out_features=128, bias=True)
      (post_mem): LIFNeuron()
      (output_linear): Linear(in_features=128, out_features=2, bias=True)
      (output): OutputNeuron()
    )
  )
  (model): DynNetwork(
    (layers): ModuleDict(
      (obs): Selector()
      (probe): Selector()
      (pre_mem_linear): Linear(in_features=130, out_features=128, bias=True)
      (pre_mem): LIFNeuron()
      (mem_linear): Linear(in_features=128, out_features=128, bias=True)
      (mem): CooldownNeuron(
        (elu): ELU(alpha=1.0)
      )
      (post_mem_linear): Linear(in_features=129, out_features=128, 

In [19]:
for p in model.model.layers['output_linear'].parameters():
    print(p.shape)

torch.Size([2, 128])
torch.Size([2])


In [18]:
model.model.layers['output_linear'].parameters()

<generator object Module.parameters at 0x7fbeccb995c8>